In [1]:
#import libraries
from bs4 import BeautifulSoup
import requests
import sqlite3
import csv

In [2]:
#get a response
url = "https://realpython.github.io/fake-jobs"
html = requests.get(url)

In [3]:
#parse html 
soup = BeautifulSoup(html.text, 'html.parser')

Task 1


In [4]:
job_list = []
for job_card in soup.find_all("div", class_="card-content"):
        title = job_card.find("h2", class_="title").get_text(strip=True)
        company = job_card.find("h3", class_="company").get_text(strip=True)
        location = job_card.find("p", class_="location").get_text(strip=True)
        description = job_card.find("div", class_="description")
        description_text = description.get_text(strip=True) if description else "No description available"
        apply_link = job_card.find("a", string="Apply")
        apply_url = apply_link["href"] if apply_link else "No link available"

        job_list.append((title, company, location, description_text, apply_url))
                



In [5]:
job_list

[('Senior Python Developer',
  'Payne, Roberts and Davis',
  'Stewartbury, AA',
  'No description available',
  'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'),
 ('Energy engineer',
  'Vasquez-Davidson',
  'Christopherville, AA',
  'No description available',
  'https://realpython.github.io/fake-jobs/jobs/energy-engineer-1.html'),
 ('Legal executive',
  'Jackson, Chambers and Levy',
  'Port Ericaburgh, AA',
  'No description available',
  'https://realpython.github.io/fake-jobs/jobs/legal-executive-2.html'),
 ('Fitness centre manager',
  'Savage-Bradley',
  'East Seanview, AP',
  'No description available',
  'https://realpython.github.io/fake-jobs/jobs/fitness-centre-manager-3.html'),
 ('Product manager',
  'Ramirez Inc',
  'North Jamieview, AP',
  'No description available',
  'https://realpython.github.io/fake-jobs/jobs/product-manager-4.html'),
 ('Medical technical officer',
  'Rogers-Yates',
  'Davidville, AP',
  'No description available',
  'https:/

In [9]:
with sqlite3.connect("jobs.db") as connection :
    cursor = connection.cursor()

    query = """
            Create table if not exists jobs(
            ID integer primary key AUTOINCREMENT,
            Title text,
            Company text,
            Location text,
            Description text,
            link text,
            
            UNIQUE (Title, Company, Location)
            );    
            """
    cursor.execute(query)

    for job in job_list:

        insert = """  
                INSERT INTO jobs (Title, Company, Location, Description, link)
                VALUES (?, ?, ?, ?, ?)
                ON CONFLICT(Title, Company, Location) 
                DO UPDATE SET Description=excluded.Description, link=excluded.link;
                """  
        cursor.execute(insert,job)

def filter(location=None, company = None):
    query = """ SELECT * FROM jobs WHERE 1=1"""
    params = []

    if location:
        query += " AND Location = ?"
        params.append(location)

    if company:
        query += " AND Company = ?"
        params.append(company)

    with sqlite3.connect("jobs.db") as connection:
        cursor = connection.cursor()

        cursor.execute(query,params)

        results = cursor.fetchall()
        return results
    
def to_csv(results, filename = "filtered_jobs.csv"):
    with open(filename,'w',newline='',encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Job Title","Company Name","Location","Description","Application Link"])
        writer.writerows(results)
    print("Exported to csv!")

results = filter()
to_csv(results)


                 
    

Exported to csv!
